# Parameter Configurations

When larvaworld is imported the registry is automatically initialized.

We import the registry module **reg**

In [ ]:
import larvaworld
from larvaworld.lib import reg, sim, util

The configuration types (conftypes) stored in the registry can be accessed easily :

Each conftype is managed by a dedicated instance of the **reg.generators.ConfType** class.

These instances are located in a dictionary accessed as **reg.conf** under the respective conftype as key.

They are easily accessed as the reg.conf is an AttrDict, 

For example, the ConfType instance responsible for a conftype :


In [ ]:
print(larvaworld.CONFTYPES)

In [ ]:
conftype = "Env"
ct = reg.conf.Env
assert ct == reg.conf[conftype]
assert ct.conftype == conftype

print(f"The ConfType for {conftype} is an instance of {ct.__class__}")

Each ConfType instance **ct** manages configurations of conftype **ct.conftype**.

A number of stored configurations are kept as entries in a dictionary  **ct.dict** stored at **ct.path_to_dict**.

Each entry has a unique ID (confID) as key. The list of available IDs cna be accessed as **ct.confIDs**.





In [ ]:
# The dictionary
print(
    f"The dictionary storing {ct.conftype} configurations is an instance of {ct.dict.__class__}"
)


# The path where the dictionary is stored:
print(f"It is stored at {ct.path_to_dict}")

In [ ]:
print("The number of stored configurations per conftype & some example IDs:")
print()
for k in larvaworld.CONFTYPES:
    ct = reg.conf[k]
    assert k == ct.conftype
    ids = ct.confIDs
    print(f" - {k} : {len(ids)}     eg : {ids[:3]}")
    # print(f'        {ids}')
    # print()

In [ ]:
ct = reg.conf.Env
id = ct.confIDs[1]


# The configuration IDs are the keys. They correspond to a configuration stored as a nested dictionary :
entry1 = ct.dict[id]
print()
print(f"An instance of {entry1.__class__}")

# The configuration can be retrieved directly by :
entry2 = ct.getID(id)
assert entry1 == entry2

# The configuration entry is a nested dict and can be printed easily as such :
print(f"The lab-specific data-format configuration stored under ID {id} is :")
entry2.print()

In [ ]:
# A new configuration can be created by altering an existing :
new_conf = entry1.get_copy()
new_conf.arena.dims = (0.5, 0.1)
print(f"Old : {entry1.arena.dims} vs New : {new_conf.arena.dims}")


# and then stored under an ID :
new_id = "new_confID"
assert new_id not in ct.confIDs
ct.setID(id=new_id, conf=new_conf)
assert new_id in ct.confIDs

# an entry can be deleted :
ct.delete(id=new_id)
assert new_id not in ct.confIDs

In [ ]:
# The configuration object can be retrieved directly by :
obj = ct.get(id)
print(f"The object under the ID : {id} is an instance of {obj.__class__}")
print()

# %params obj

In [ ]:
%load_ext param.ipython
from param.ipython import ParamPager
import panel as pn

In [ ]:
from larvaworld.lib.reg.generators import EnvConf

%params EnvConf

In [ ]:
for id in ct.confIDs:
    obj = ct.get(id)
    # %params obj

In [ ]:
obj = ct.get(ct.confIDs[2])
%params obj.odorscape

In [ ]:
for id in ct.confIDs:
    obj = ct.get(id)
    obj.visualize(duration=0.3)

In [ ]:
for id in ct.confIDs:
    obj = ct.get(id)
    print(id)
    p = util.AttrDict({"env_params": obj.nestedConf})

    m = sim.base_run.BaseRun(
        runtype="Exp",
        experiment="dish",
        parameters=p,
        id=obj.name,
        duration=0.3,
        screen_kws={
            "show_display": True,
            "mode": "video",
            "odor_aura": True,
            "intro_text": False,
            "fps": 60,
        },
    )
    m.build_env(m.p.env_params)
    m.set_obj_visibility(m.sensorscapes, True)
    m.run()
    m.screen_manager.close()